## Import all the libraries required

In [52]:
# Import all the libraries that is required for AIChatbot
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence

from sklearn.preprocessing import LabelEncoder
import numpy
import pandas as pd
import numpy as np
from sklearn import linear_model, datasets

from termcolor import colored
# fix random seed for reproducibility
numpy.random.seed(7)


## Import all the data required

In [106]:
#import data
DirectAnswer_DB = pd.read_csv("C:\\Users\\Ashlesh B Shetty\\Google Drive\\LaptopOnDrive\\SpringSemester\\ELP\\EndToEndChatBot\\Data\\DirectAnswer_DB.csv")
RDBMS_MeetingRoomTable = pd.read_csv("C:\\Users\\Ashlesh B Shetty\\Google Drive\\LaptopOnDrive\\SpringSemester\\ELP\\EndToEndChatBot\\Data\\RDBMS_MeetingRoomTable.csv")
TrainigData_SentimentAnalysisForSensitivity = pd.read_csv("C:\\Users\\Ashlesh B Shetty\\Google Drive\\LaptopOnDrive\\SpringSemester\\ELP\\EndToEndChatBot\\Data\\TrainigData_SentimentAnalysisForSensitivity.csv")
IntentEntity_Action_Mapper = pd.read_csv("C:\\Users\\Ashlesh B Shetty\\Google Drive\\LaptopOnDrive\\SpringSemester\\ELP\\EndToEndChatBot\\Data\\IntentEntity_Action_Mapper.csv")
TrainingData_EntityIntent = pd.read_csv("C:\\Users\\Ashlesh B Shetty\\Google Drive\\LaptopOnDrive\\SpringSemester\\ELP\\EndToEndChatBot\\Data\\TrainingData_EntityIntent_v2.csv")
Entities_BagOfWords = pd.read_csv("C:\\Users\\Ashlesh B Shetty\\Google Drive\\LaptopOnDrive\\SpringSemester\\ELP\\EndToEndChatBot\\Data\\Entities_BagOfWords.csv", header = None)

In [4]:
DirectAnswer_DB

,Intent,Entity1,Entity2,Action,ActionSubclass,Answer
0,Who Or What is it?,attend,orientation,Direct,Low Entropy,"It is not required, but highly encouraged for ..."
1,Who Or What is it?,receive,paycheck,Direct,Low Entropy,Your first paycheck will be deposited the last...
2,Who Or What is it?,upload,timesheet,Direct,Low Entropy,Please submit your timesheet on our timesheet ...
3,Who Or What is it?,check,deposit,Direct,Low Entropy,We need official documentation of your routing...
4,Who Or What is it?,it,ticket,Knowledgebase,Medium Entropy,Please refer to out knowledge base article htt...


## Initializing parameters that will be used later

In [5]:
max_review_length = 22
# count_Questions = 0
# count_Answered_Satisfactorily = 0

##Sensitive
# QuestionAsked = "What is my salary compensation structure?"

##Cannot parse Intent/Entity
# QuestionAsked = "When is the meeting?"

## Questions that bot can answer directly? (Low Entropy Answers)
# QuestionAsked = "Am I required to attend the New Employee Welcome and Benefits Orientation program?"

## Questions for which bot directs to Knowledgebase? (Low Entropy Answers)
# QuestionAsked = "How do I open an IT ticket if I have an issue?"

## Questions for which bot queries a inhouse database to fetch the answers? (Low Entropy Answers)
# QuestionAsked = "Which is the largest conference room that I can book?"

## Questions for which crowdsource option is provided? (Low Entropy Answers)
# QuestionAsked = "Is there any good movie suggestions for consultants?"

## Questions for which crowdsource option is not provided since PDM is only the right person to answer? (Low Entropy Answers)
# QuestionAsked = "Who are my team members and what are their roles?"

## Questions for which both entity and intent is parsed but action is not available
# QuestionAsked = "What do I do if I have to travel before I get my AMEX card?"
# You will need to use your personal credit card and you will then be reimbursed. The instructions on how to receive reimbursement can be found with your Offer 

## Build a logistic regression model for senstitivity check

In [6]:
# Batchprocess trained Losgistic regression model for Sensitivity scoring 
questions  = TrainigData_SentimentAnalysisForSensitivity['Question']
t1 = Tokenizer()
t1.fit_on_texts(questions)
question_docs = numpy.asarray(t1.texts_to_sequences(questions))
max_review_length = 22
logis_train = sequence.pad_sequences(question_docs, maxlen=max_review_length)
#print(logis_train)
logis_y = TrainigData_SentimentAnalysisForSensitivity['SensitiveFlag']
#print(logis_y)

logreg = linear_model.LogisticRegression(C=1e5)
# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(logis_train, logis_y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

## Function that checks sensitivity

In [7]:
def sensitivity(QuestionAsked):
    words = text_to_word_sequence(QuestionAsked)
    text1 = numpy.array([t1.word_index[word] if word in t1.word_index else 0 for word in words])
    text2 = sequence.pad_sequences([text1], maxlen=max_review_length)
    return logreg.predict(text2)[0]

## Training the RNN-LSTM model

In [9]:
# Training Data Processing
## create indipendent variable array
numpy.random.seed(7)
TrainingData_EntityIntent = pd.read_csv("C:\\Users\\Ashlesh B Shetty\\Google Drive\\LaptopOnDrive\\SpringSemester\\ELP\\EndToEndChatBot\\Data\\TrainingData_EntityIntent_v2.csv")
docs  = TrainingData_EntityIntent['Question']
t = Tokenizer()
t.fit_on_texts(docs)
encoded_docs = numpy.asarray(t.texts_to_sequences(docs))
max_review_length = 22
X_train = sequence.pad_sequences(encoded_docs, maxlen=max_review_length)
#print(X_train[:3,])

## Dependent variable data processing
Y = TrainingData_EntityIntent['Intent']
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)
#print(dummy_y[:3])
    
## Setup the neural network layers
numpy.random.seed(7)
embedding_vecor_length = 32
top_words = 5000
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#print(model.summary())

model.fit(X_train, dummy_y, nb_epoch=5, batch_size=64)

C:\Users\Ashlesh B Shetty\Anaconda2\envs\starterbot\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/5
22/22 [==============================] - 20s 920ms/step - loss: 0.6934 - acc: 0.4091
Epoch 2/5
22/22 [==============================] - 20s 907ms/step - loss: 0.6915 - acc: 0.6364
Epoch 3/5
22/22 [==============================] - 20s 923ms/step - loss: 0.6894 - acc: 0.8636
Epoch 4/5
22/22 [==============================] - 20s 911ms/step - loss: 0.6871 - acc: 0.9318
Epoch 5/5
22/22 [==============================] - 20s 897ms/step - loss: 0.6845 - acc: 0.9545


## Function for Intent parsing

In [10]:
def intent_parsing(QuestionAsked):
    words = text_to_word_sequence(QuestionAsked)
    text1 = numpy.array([t.word_index[word] if word in t.word_index else 0 for word in words])
    text2 = sequence.pad_sequences([text1], maxlen=max_review_length)
    # print(model.predict(text2))
    # print(model.predict_classes(text2)[0])
    return encoder.classes_[model.predict_classes(text2)[0]]

## Function for entity parsing

In [11]:
def entity_parsing(QuestionAsked):
    words = text_to_word_sequence(QuestionAsked)
    return [i for i in filter(lambda X: X in words, Entities_BagOfWords[0].tolist())]

## Function for Decision module

In [12]:
def action_decision_module(QuestionAsked):
    QuestionAsked_EntInt =  [intent_parsing(QuestionAsked)] + entity_parsing(QuestionAsked)[0:2]
    
    for (intent , entity1 , entity2, action,ActionSubclass ) in zip(IntentEntity_Action_Mapper['Intent'], 
                                                                    IntentEntity_Action_Mapper['Entity1'],
                                                                    IntentEntity_Action_Mapper['Entity2'],
                                                                    IntentEntity_Action_Mapper['Action'],
                                                                    IntentEntity_Action_Mapper['ActionSubclass']):
        action_actionsubclass = (intent , entity1 , entity2, action, ActionSubclass)
#         print(action_actionsubclass)
#         print(QuestionAsked_EntInt)
        if set(QuestionAsked_EntInt) == set([intent, entity1, entity2]):
            action_taken = action_actionsubclass
            break
        else:
            action_taken = ("NA","NA", "NA","NA", "NA")
        
        if ((len(QuestionAsked_EntInt) ==3) & 
            (action_taken == ("NA","NA", "NA","NA", "NA"))):
            action_taken = (QuestionAsked_EntInt[0],QuestionAsked_EntInt[1], QuestionAsked_EntInt[2],"NA", "PDM_LearningOption")
        
    return action_taken
# intent_parsing(QuestionAsked)
# entity_parsing(QuestionAsked)
# action_decision_module(QuestionAsked)

## Function for the Answering module

In [11]:
# action_decision_module(QuestionAsked)

## Feedback and ChatStats aggregrator module

In [60]:
def feedback_chatstats_aggregrator():
    global count_Questions
    count_Questions  += 1
    global count_Answered_Satisfactorily
    
    print("Was I able to answer your question? (yes/no)")
    print(colored('User:', 'green',attrs=['bold']))
    feedback = input("")
    
    if feedback.lower() == "yes":
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Awesome! Happy to help.")
        count_Answered_Satisfactorily  += 1
        
    else:
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Oh. Sorry. I might have misunderstood your question. Let me ask your PDM. \nYou will hear from your PDM soon.")

# feedback_chatstats_aggregrator()

In [75]:
def answering_module(QuestionAsked):
    action_taken = action_decision_module(QuestionAsked)
    global DirectAnswer_DB
    
    if ((action_taken[3] == 'Direct')  & 
        (action_taken[4] == 'Low Entropy')):    
        answer_record = DirectAnswer_DB[(DirectAnswer_DB['Intent'] == action_taken[0]) & 
                                 (DirectAnswer_DB['Entity1'] == action_taken[1]) &
                                 (DirectAnswer_DB['Entity2'] == action_taken[2]) & 
                                 (DirectAnswer_DB['Action'] == action_taken[3]) & 
                                 (DirectAnswer_DB['ActionSubclass'] == action_taken[4])]
        answer = answer_record.iloc[0]['Answer']
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print(answer)
        feedback_chatstats_aggregrator()

    if ((action_taken[3] == 'Knowledgebase') & 
        (action_taken[4] == 'Medium Entropy')):    
        answer_record = DirectAnswer_DB[(DirectAnswer_DB['Intent'] == action_taken[0]) & 
                                 (DirectAnswer_DB['Entity1'] == action_taken[1]) &
                                 (DirectAnswer_DB['Entity2'] == action_taken[2]) & 
                                 (DirectAnswer_DB['Action'] == action_taken[3]) & 
                                 (DirectAnswer_DB['ActionSubclass'] == action_taken[4])]
        answer = answer_record.iloc[0]['Answer']
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Here is a link that might help answer your question")
        print(answer)
        feedback_chatstats_aggregrator()
        
    if ((action_taken[3] == 'Query') & 
        (action_taken[4] == 'Medium Entropy')):
        maxsize = RDBMS_MeetingRoomTable.Size.max()  
        answer_record = RDBMS_MeetingRoomTable[(RDBMS_MeetingRoomTable['Size'] == maxsize) &
                                                (RDBMS_MeetingRoomTable['RoomType']== 'conference')]
        answer = answer_record.iloc[0]['RoomAddress']
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Here is the answer that I got after querying for your question:")
        print(answer)
        feedback_chatstats_aggregrator()
        
    if ((action_taken[3] == 'Crowdsource') & 
        (action_taken[4] == 'High Entropy (opinions,suggestions)')):
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("This is a type of question that I recommend to crowdsource. Do you want to crowdsource it or ask your PDM?")
        user_input = input('')
        if user_input.lower() == "crowdsource":
            print(colored('Chatbot:', 'red',attrs=['bold']))
            print("Your question has been posted on crowdsourcing platform. You will soon get a response.")
            print("\n \n \n \n")
            print(colored('CROWDSOURCING PLATFORM:', 'blue',attrs=['bold']))
            print(colored('Chatbot:', 'red',attrs=['bold']))
            print("Hello people, can anyone help me answer this question:" + QuestionAsked)
            crowdsourced_ans1 = input('')
            crowdsourced_ans2 = input('')
            print("\n \n \n \n")
            print(colored('BACK TO USER CHAT PLATFORM', 'blue',attrs=['bold']))
            print(colored('Chatbot:', 'red',attrs=['bold']))
            print("Hello, here are few crowdsourced answers to your question \n" + 
                  crowdsourced_ans1 + "\n" +
                 crowdsourced_ans2)
            feedback_chatstats_aggregrator()
            
        else:
            print(colored('Chatbot:', 'red',attrs=['bold']))
            print(" Thank you for your feedback. Let me ask you PDM. You will hear back from your PDM soon")
        

    if ((action_taken[3] == 'NA') & 
        (action_taken[4] == 'PDM_LearningOption')):
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("I get your question but unfortunately I don't have an answer for this. \nLet me ask your PDM. You will hear from your PDM soon")
        print(" \n \n \n \n")
        print(colored('PDM CHAT PLATFORM:', 'blue',attrs=['bold']))
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Hello, Can you help me answer this question that a new consulstant asked me? I could not think of an answer for this question:\n" + QuestionAsked)
        print(colored('PDM:', 'green',attrs=['bold']))
        answer = input('')
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Do you want the me to learn this answer and directly answer to similar question if asked again?")
        print(colored('PDM:', 'green',attrs=['bold']))
        PDM_decision = input('')
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Cool.")
        if PDM_decision.lower() == 'yes':
            
            #Learning module
            
            IEAM_file_update = pd.DataFrame.from_dict({"Intent": action_taken[0],
             "Entity1": action_taken[1],
             "Entity2": action_taken[2],
             "Action": ["Direct"],
             "ActionSubclass": ["Low Entropy"]})
            IEAM_file_update = IEAM_file_update[['Intent', 'Entity1', 'Entity2', 'Action', 'ActionSubclass']]
            global IntentEntity_Action_Mapper
            IntentEntity_Action_Mapper = IntentEntity_Action_Mapper.append(IEAM_file_update)

            DADB_file_update = pd.DataFrame.from_dict({"Intent": action_taken[0],
             "Entity1": action_taken[1],
             "Entity2": action_taken[2],
             "Action": ["Direct"],
             "ActionSubclass": ["Low Entropy"],
             "Answer": answer})
            DADB_file_update = DADB_file_update[['Intent', 'Entity1', 'Entity2', 'Action', 'ActionSubclass','Answer']]
            DirectAnswer_DB = DirectAnswer_DB.append(DADB_file_update)

        print(" \n \n \n \n")
        print(colored('BACK TO USER CHAT PLATFORM', 'blue',attrs=['bold']))
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("Hello, \n" + answer)
        

    
    if ((action_taken[3] == 'PDM') & 
        (action_taken[4] == 'PDM Best Person To Answer')):
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("PDM is the best person to answer this type of question. Let me ask your PDM. You will hear from your PDM soon")
        
    
   
# answering_module(QuestionAsked)

In [57]:
count_Questions = 0
count_Answered_Satisfactorily = 0

def AI_CHATBOT():
    print(colored('Chatbot:', 'red',attrs=['bold']))
    print("Hello, How can I help you ?")
    print(colored('User:', 'green',attrs=['bold']))
    QuestionAsked = input('')
    if sensitivity(QuestionAsked) == 1:
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print("This is a type of question which I actually redirect to PDM. Let me ask your PDM. \n You will hear back from your PDM soon.")

    elif (len([intent_parsing(QuestionAsked)] + entity_parsing(QuestionAsked)[0:2]) < 3):
        print(colored('Chatbot:', 'red',attrs=['bold']))
        print(" I'm a little embarrassed about this, but I don't understand your question fully. Let me ask your PDM. \nYou will hear back from your PDM soon.")

    else:
        answering_module(QuestionAsked)

# Wecome to AI-Chatbot Protoype Demo

## Case1: Sensitive
"What is my salary compensation structure?"

In [107]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
What is my salary compensation structure?
Chatbot:
This is a type of question which I actually redirect to PDM. Let me ask your PDM. 
 You will hear back from your PDM soon.


## Case2: Questions that bot can answer directly? (Low Entropy Answers)
"Am I required to attend the New Employee Welcome and Benefits Orientation program?"

In [108]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
Am I required to attend the New Employee Welcome and Benefits Orientation program?
Chatbot:
It is not required, but highly encouraged for full-time employees.  You will gain valuable information and resources that will assist you in becoming acclimated to the company.
Was I able to answer your question? (yes/no)
User:
yes
Chatbot:
Awesome! Happy to help.


## Case3-A: Questions for which both entity and intent is parsed but action is not available
"What do I do if I have to travel before I get my AMEX card?"

"You will need to use your personal credit card and you will then be reimbursed. The instructions on how to receive reimbursement can be found with your Offer"

In [109]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
What do I do if I have to travel before I get my AMEX card?
Chatbot:
I get your question but unfortunately I don't have an answer for this. 
Let me ask your PDM. You will hear from your PDM soon
 
 
 
 

PDM CHAT PLATFORM:
Chatbot:
Hello, Can you help me answer this question that a new consulstant asked me? I could not think of an answer for this question:
What do I do if I have to travel before I get my AMEX card?
PDM:
You will need to use your personal credit card and you will then be reimbursed. The instructions on how to receive reimbursement can be found with your Offer
Chatbot:
Do you want the me to learn this answer and directly answer to similar question if asked again?
PDM:
yes
Chatbot:
Cool.
 
 
 
 

BACK TO USER CHAT PLATFORM
Chatbot:
Hello, 
You will need to use your personal credit card and you will then be reimbursed. The instructions on how to receive reimbursement can be found with your Offer


## Case3-B: LEARNED: Chatbot learns based on PDM's decisions and inputs
"What do I do if I have to travel before I get my AMEX card?"

In [110]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
What do I do if I have to travel before I get my AMEX card?
Chatbot:
You will need to use your personal credit card and you will then be reimbursed. The instructions on how to receive reimbursement can be found with your Offer
Was I able to answer your question? (yes/no)
User:
yes
Chatbot:
Awesome! Happy to help.



## Case4: Questions for which bot directs to Knowledgebase? (Medium Entropy Answers)
"How do I open an IT ticket if I have an issue?"

In [111]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
How do I open an IT ticket if I have an issue?
Chatbot:
Here is a link that might help answer your question
Please refer to out knowledge base article https://goo.gl/UwKdVW
Was I able to answer your question? (yes/no)
User:
yes
Chatbot:
Awesome! Happy to help.


## Case5: Questions for which crowdsource option is provided? (High Entropy Answers)
"Is there any good movie suggestions for consultants?"

In [112]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
Is there any good movie suggestions for consultants?
Chatbot:
This is a type of question that I recommend to crowdsource. Do you want to crowdsource it or ask your PDM?
crowdsource
Chatbot:
Your question has been posted on crowdsourcing platform. You will soon get a response.

 
 
 

CROWDSOURCING PLATFORM:
Chatbot:
Hello people, can anyone help me answer this question:Is there any good movie suggestions for consultants?
Wolf of Wall Street
Pursuit of Happyness

 
 
 

BACK TO USER CHAT PLATFORM
Chatbot:
Hello, here are few crowdsourced answers to your question 
Wolf of Wall Street
Pursuit of Happyness
Was I able to answer your question? (yes/no)
User:
yes
Chatbot:
Awesome! Happy to help.


## Case6: Questions for which crowdsource option is not provided since PDM is only the right person to answer?
"Who are my team members and what are their roles?"

In [113]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
Who are my team members and what are their roles?
Chatbot:
PDM is the best person to answer this type of question. Let me ask your PDM. You will hear from your PDM soon


## Case7: Cannot parse Intent/Entity
"When is the meeting?"

In [114]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
When is the meeting?
Chatbot:
 I'm a little embarrassed about this, but I don't understand your question fully. Let me ask your PDM. 
You will hear back from your PDM soon.


## Case8: Questions for which bot queries a inhouse database to fetch the answers? (Medium Entropy Answers)
"Which is the largest conference room that I can book?"

In [115]:
AI_CHATBOT()

Chatbot:
Hello, How can I help you ?
User:
Which is the largest conference room that I can book?
Chatbot:
Here is the answer that I got after querying for your question:
Block A - L3 - AL3R12
Was I able to answer your question? (yes/no)
User:
no
Chatbot:
Oh. Sorry. I might have misunderstood your question. Let me ask your PDM. 
You will hear from your PDM soon.
